In [ ]:
%matplotlib inline

In [ ]:
import os, math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

In [ ]:
# safe the data from the GCAL tutorial notebook:

"""import numpy
data = topo.sim.views              # The measurement data saved in the snapshot

pref = data.OrientationPreference.V1
sel =  data.OrientationSelectivity.V1
v1resp = line.V1

pref_data = pref.last.data
sel_data = sel.last.data
v1resp_data = v1resp.last.data
numpy.savetxt("pref_data.txt", pref_data)
numpy.savetxt("sel_data.txt", sel_data)
numpy.savetxt("v1resp_data.txt", v1resp_data)"""

# and import it here
pref_data = np.loadtxt("pref_data.txt")
sel_data = np.loadtxt("sel_data.txt")
v1resp_data = np.loadtxt("v1resp_data.txt")

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

In [ ]:
# transform angles into vectors
# X, Y -  coordinates for plotting
# UN - the actual data
# UV - hell knows, but used for colormap

X,Y = np.meshgrid(np.arange(0, pref_data.shape[0]), np.arange(0, pref_data.shape[1]))

UN = np.zeros((47, 47))
VN = np.zeros((47, 47))

for i in range(47):
    for j in range(47):
        UN[i,j] = np.cos(pref_data[i, j])
        VN[i,j] = np.sin(pref_data[i, j])

# because of the weird coordinates (from 0 to 47 for X-axis and from 47 to 0 for Y-axis), need to flip the data
Y = -Y
UN = -UN
VN = -VN



In [ ]:
# plot the initial map and vector field
# somehow the vector field gets shifted - fix it

plt.figure()
plt.subplot(2, 2, 1)
plt.imshow(UN, extent=[X.min(), X.max(), Y.min(), Y.max()], cmap=cm.hsv)
plt.colorbar()
plt.title('Orientation map')

plt.subplot(2, 2, 2)
plt.quiver(X, Y, UN, VN, 
           color='Teal', 
           headlength=7)

plt.title('Vector Field')
plt.show()

In [ ]:
# plot the initial orientation map and the vector field on it

skip = (slice(None, None, 2), slice(None, None, 2)) # to make the vector field less dense

fig, ax = plt.subplots()
plt.imshow(UN, extent=[X.min(), X.max(), Y.min(), Y.max()], cmap=cm.hsv)
plt.colorbar()
plt.quiver(X[skip], Y[skip], UN[skip], VN[skip])


plt.title('Orientation map with the vector field')
plt.show()

In [ ]:
# get rid of the background image and plot just the vector field but colored for better overview 

# rcParams['figure.figsize'] = 15, 10 # adjust the size of figures

plt.clf()
plt.figure()


plt.subplot(2, 2, 1)
plt.quiver(X[skip], Y[skip], UN[skip], VN[skip],    # data
           pref_data,       # colour the arrows based on this array
           cmap=cm.hsv)     # colour map


plt.colorbar()                 # adds the colour bar

plt.title('Vector field and colormap - orientation map')



plt.subplot(2, 2, 2)
plt.quiver(X[skip], Y[skip], UN[skip], VN[skip],    # data
           sel_data,        # colour the arrows based on this array -> selectivity map
           cmap=cm.hsv)     # colour map

plt.colorbar()
plt.title('Vector field - orientation map; Colormap - selectivity')


plt.show()

In [ ]:
# play around with colours

import matplotlib.colors as col
import seaborn as sns
import scipy

rcParams['figure.figsize'] = 20, 13

##### generate custom colormap
def mkcmap(): 
    white = '#ffffff'
    black = '#000000'
    red = '#ff0000'
    blue = '#0000ff'
    green = '#00ff00'
    
    first = '#641E16'
    second = '#A93226'
    third = '#C70039'
    fourth = '#FF5733'
    last = '#FFC300'
    anglemap = col.LinearSegmentedColormap.from_list(
        'anglemap', [black, '#4d3900', '#997300',  '#e6ac00', '#ffcc33', '#ffdf80', white, '#ffdf80', '#ffcc33', '#e6ac00', '#997300', '#4d3900', black], N=256, gamma=1)
    return anglemap

anglemap = mkcmap()
#huslmap = col.ListedColormap(sns.color_palette("husl",256))

##### generate data grid
x = X
y = Y
z = pref_data

##### plot with different colormaps
plt.figure()
plt.clf()

plt.subplot(2,2,1)
plt.pcolormesh(X, Y, pref_data, 
    cmap=anglemap,vmin=0, vmax=np.pi)
plt.axis([X.min(), X.max(), Y.min(), Y.max()])
plt.colorbar()
#cbar.ax.set_ylabel('Phase [pi]')
plt.title('Dominant (black/horizontal and white/vertical) vs non-dominant (yellow/diagonal) orientation')

plt.subplot(2,2,2)
plt.pcolormesh(X, Y, pref_data, 
    cmap=plt.get_cmap('Spectral'),vmin=0, vmax=np.pi)
plt.axis([X.min(), X.max(), Y.min(), Y.max()])
plt.title('Spectral colormap')
plt.colorbar()

plt.subplot(2,2,3)
plt.pcolormesh(X, Y, pref_data, 
    cmap=plt.get_cmap('RdGy'),vmin=0, vmax=np.pi)
plt.axis([X.min(), X.max(), Y.min(), Y.max()])
plt.title('diverging colormap')
plt.colorbar()

plt.subplot(2,2,4)
plt.pcolormesh(X, Y, pref_data, 
    cmap=plt.get_cmap('BrBG'),vmin=0, vmax=np.pi)
plt.axis([X.min(), X.max(), Y.min(), Y.max()])
plt.title('diverging colormap')
plt.colorbar()

plt.show()

In [ ]:
# function to define the gradient

def map_gradient(data):
    orient_map_gradient_X, orient_map_gradient_Y = np.gradient(data)
    #plt.quiver(orient_map_gradient_X, orient_map_gradient_Y)
    orient_map_gradient = np.zeros((orient_map_gradient_X.shape[0], orient_map_gradient_X.shape[0]))
    for i in xrange(orient_map_gradient[0].shape[0]):
        for j in xrange(orient_map_gradient[0].shape[0]):
            orient_map_gradient[i, j] = np.sqrt(orient_map_gradient_X[i, j]**2 + orient_map_gradient_X[i, j]**2)
    return orient_map_gradient

orient_map_gradient = map_gradient(pref_data)
sel_map_gradient = map_gradient(sel_data)

In [ ]:
plt.clf()
plt.subplot(2, 2, 1)
plt.imshow(pref_data, interpolation="none", cmap=plt.get_cmap('RdGy'))
plt.title("Orientation map")
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(sel_data, interpolation="none", cmap=cm.jet)
plt.title("Selectivity map")
plt.colorbar()

plt.subplot(2, 2, 3)
plt.imshow(orient_map_gradient, interpolation="none")
plt.title("Orientation map gradient")
plt.subplot(2, 2, 4)
plt.imshow(sel_map_gradient, interpolation="none")
plt.title("Selectivity map gradient")
plt.show()


In [ ]:
import matplotlib.colors.g

In [ ]:
# more colours and contours

#rcParams['figure.figsize'] = 20, 13

#!/usr/bin/env python
'''
Test combinations of contouring, filled contouring, and image plotting.
For contour labelling, see contour_demo.py.

The emphasis in this demo is on showing how to make contours register
correctly on images, and on how to get both of them oriented as
desired.  In particular, note the usage of the "origin" and "extent"
keyword arguments to imshow and contour.
'''
from matplotlib import mlab, cm

# Default delta is large because that makes it fast, and it illustrates
# the correct registration between image and contours.
delta = 0.3
extent = (0, 47, 0, 47)

x = UN 
y = VN 
Z = UN


levels = np.arange(-2.0, 1.601, 0.4)  # Boost the upper limit to avoid truncation errors.

norm = cm.colors.Normalize(vmax=abs(Z).max(), vmin=-abs(Z).max())
cmap = cm.jet

plt.figure()


plt.subplot(2, 2, 1)

cset1 = plt.contourf(X, Y, Z, levels,
                 cmap=cm.get_cmap(cmap, len(levels) - 1),
                 norm=norm,
                 )
# It is not necessary, but for the colormap, we need only the
# number of levels minus 1.  To avoid discretization error, use
# either this number or a large number such as the default (256).

# If we want lines as well as filled regions, we need to call
# contour separately; don't try to change the edgecolor or edgewidth
# of the polygons in the collections returned by contourf.
# Use levels output from previous call to guarantee they are the same.
cset2 = plt.contour(X, Y, Z, cset1.levels,
                colors='k',
                hold='on')
# We don't really need dashed contour lines to indicate negative
# regions, so let's turn them off.
for c in cset2.collections:
    c.set_linestyle('solid')

# It is easier here to make a separate call to contour than
# to set up an array of colors and linewidths.
# We are making a thick green line as a zero contour.
# Specify the zero level as a tuple with only 0 in it.
cset3 = plt.contour(X, Y, Z, (0,),
                colors='g',
                linewidths=2,
                hold='on')
plt.title('Filled contours')
plt.colorbar(cset1)



plt.show()

In [ ]:
# more contours

fig = plt.figure(1)
plt.clf()

plt.subplot(2,2,1)

# set desired contour levels.
clevs = np.arange(960,1061,5)

# plot SLP contours.
CS1 = plt.contour(X,Y, UN, clevs,linewidths=0.5,colors='k', hold='on')
CS2 = plt.contourf(X,Y,UN, clevs,cmap=plt.cm.RdBu_r, hold='on')


# now plot.
c = plt.pcolor(X,Y,UN)
d = plt.colorbar(c)
q = plt.hsv()       # colormap
e = plt.contour(X,Y,UN,colors='k')


plt.show()


In [ ]:
# histogramms

import matplotlib.cm

# orientation
n_bins = 10

fig = plt.figure()
ax = fig.add_axes([0.1,0.1, 0.8,0.8], polar=True)
#color_vector = [matplotlib.cm.jet(val) for val in np.linspace(0., 1., n_bins/2)] + [matplotlib.cm.jet(val) for val in np.linspace(0., 1., n_bins/2)]

n, bins = np.histogram(pref_data.flatten(), bins=10)
#n_bins = bins.shape[0] - 1
pretend_bins = np.linspace(0., 2*np.pi, n_bins)
ax.bar(bins[:n_bins], n, (bins[1] - bins[0]))
plt.title('Orientation preference histogram')
# I really like this histogram! need to find a way toplot only half of the circle 
# and colour the bars according to the selectivity 
plt.show()


#selectivity
fig.add_subplot()
plt.hist(sel_data.flatten(), bins=25)
plt.title('Selectivity histogram')
plt.show()



In [ ]:
# orientation and selectivity combined histogram

rcParams['figure.figsize'] = 35, 13

extent = (0, 47, 0, 47)
ax = plt.subplot(1,2,1)
ax1 = plt.plot(pref_data.flatten(), sel_data.flatten()*40, 'r.', label='Selectivity')
ax.set_xlim(0, 3.15)
plt.title("Selectivity and Orientation preference")
plt.xlabel("Orientation")


#plt.subplot(1,2,2)
plt.hist(pref_data.flatten(), label='Orientation', bins = 35)

y_mean = [np.mean(sel_data.flatten())*60 for i in pref_data.flatten()]
plt.plot(pref_data.flatten(),y_mean, label='Mean Selectivity', linestyle='--')
plt.legend()
plt.show()


In [ ]:
# Fourier power spectrum = I have no idea what those colours mean

rcParams['figure.figsize'] = 12, 8

ax = plt.subplot()
sp = np.fft.fft2(UN)
#plt.plot(sp)
plt.plot(sp)
ax.set_xlim(0,46)
plt.title ('The Fourier power spectrum')

plt.show()

#As another example, an interesting property of orientation maps measured in animals is that their Fourier power 
#spectrums usually show a ring shape, because the orientations repeat at a constant spatial frequency in all 
#directions.

In [ ]:
import sys
import os
import platform

# Mindboogle

mindbog=os.path.join('/Users/ghfc/Documents/Dropbox/hugo_is_dead/scripts/mindboggle-master/')
sys.path.append(mindbog)
sys.path.append(os.path.join(mindbog,'mindboggle/shapes'))



In [ ]:
import laplace_beltrami as LB
import numpy
import scipy
import time

In [ ]:

A,B=LB.computeAB(sel_data.flatten(), pref_data.flatten())
#A and B will be used to compute laplace beltrami eigenfunstions or fourier modes